In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error,r2_score

df = pd.read_csv('resources/train.csv')
df_t = pd.read_csv('resources/test.csv')
df_t.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [3]:
#pre_process function train and test dataframes
def pre_process(df):
    df=df.drop(['MiscFeature','PoolQC','Fence','Alley'],axis=1)
    num_df=df.select_dtypes(include='number')
    num_cols=num_df.columns
    for col in num_cols:
        df[col]=df[col].fillna(df[col].median())
    str_df=df.select_dtypes(include='object')
    str_cols=str_df.columns
    for col in str_cols:
        df[col]=df[col].fillna(df['LotShape'].value_counts().idxmax())
    for col in str_cols:
        df[col]=pd.factorize(df[col])[0]  
    return df     

ptr_df=pre_process(df)
ptr_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,1,...,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,1,0,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,1,0,0,2,...,272,0,0,0,0,2,2006,0,1,140000
4,5,60,0,84.0,14260,0,1,0,0,1,...,0,0,0,0,0,12,2008,0,0,250000


In [4]:
pts_df=pre_process(df_t)
pts_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,0,80.0,11622,0,0,0,0,0,...,0,0,0,120,0,0,6,2010,0,0
1,1462,20,1,81.0,14267,0,1,0,0,1,...,36,0,0,0,0,12500,6,2010,0,0
2,1463,60,1,74.0,13830,0,1,0,0,0,...,34,0,0,0,0,0,3,2010,0,0
3,1464,60,1,78.0,9978,0,1,0,0,0,...,36,0,0,0,0,0,6,2010,0,0
4,1465,120,1,43.0,5005,0,1,1,0,0,...,82,0,0,144,0,0,1,2010,0,0


In [5]:
X=ptr_df.drop('SalePrice',axis=1)


In [8]:
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,0,65.0,8450,0,0,0,0,0,...,61,0,0,0,0,0,2,2008,0,0
1,2,20,0,80.0,9600,0,0,0,0,1,...,0,0,0,0,0,0,5,2007,0,0
2,3,60,0,68.0,11250,0,1,0,0,0,...,42,0,0,0,0,0,9,2008,0,0
3,4,70,0,60.0,9550,0,1,0,0,2,...,35,272,0,0,0,0,2,2006,0,1
4,5,60,0,84.0,14260,0,1,0,0,1,...,84,0,0,0,0,0,12,2008,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,0,62.0,7917,0,0,0,0,0,...,40,0,0,0,0,0,8,2007,0,0
1456,1457,20,0,85.0,13175,0,0,0,0,0,...,0,0,0,0,0,0,2,2010,0,0
1457,1458,70,0,66.0,9042,0,0,0,0,0,...,60,0,0,0,0,2500,5,2010,0,0
1458,1459,20,0,68.0,9717,0,0,0,0,0,...,0,112,0,0,0,0,4,2010,0,0


In [ ]:
X.head()
y=df['SalePrice']

In [6]:
model=GradientBoostingRegressor(n_estimators=700)
model.fit(X,y)
pred=model.predict(X)
mape=mean_absolute_percentage_error(y,pred)
r2=r2_score(y,pred)
print(mape,r2)

0.02521885111945754 0.9957887271432251


In [10]:
X + pred

ValueError: Unable to coerce to Series, length must be 76: given 1460

In [7]:
ts_pred=model.predict(pts_df)
end_pred=pd.DataFrame(ts_pred,index=df_t.index)
end_pred.columns=['SalePrice']
end_pred.to_csv('submission.csv',sep=',')
end_pred.head()

,SalePrice
0,136247.960095
1,167169.435432
2,190437.431873
3,183282.972257
4,192115.507457
